In [5]:
import numpy as np
import pandas as pd

This notebook will serve as an example to showcase the different flows we will implement. we will generate a dataset of 100 enumerators and 2000 targets and simulate the optimization.

# Min distance flow with haversine

In [6]:
# Set a random seed for reproducibility
np.random.seed(0)

# Generate 100 enumerators with random latitudes and longitudes
enumerator_data = {
    "enum_id": [f"E{i+1:03d}" for i in range(100)],  # Creates IDs like E001, E002, ...
    "enum_lat": np.random.uniform(-90, 90, 100),  # Random latitudes
    "enum_long": np.random.uniform(-180, 180, 100),  # Random longitudes
}

# Generate 2000 targets with random latitudes and longitudes
target_data = {
    "target_id": [
        f"T{i+1:04d}" for i in range(2000)
    ],  # Creates IDs like T0001, T0002, ...
    "target_lat": np.random.uniform(-90, 90, 2000),  # Random latitudes
    "target_long": np.random.uniform(-180, 180, 2000),  # Random longitudes
}

# Create the DataFrames
df_enum = pd.DataFrame(enumerator_data)
df_target = pd.DataFrame(target_data)

# Display the shape of the created DataFrames to confirm the number of rows
print(f"Enumerators DataFrame shape: {df_enum.shape}")
print(f"Targets DataFrame shape: {df_target.shape}")

Enumerators DataFrame shape: (100, 3)
Targets DataFrame shape: (2000, 3)


In [7]:
from surveyscout.utils import LocationDataset

enum_locations = LocationDataset(df_enum, "enum_id", "enum_lat", "enum_long")
target_locations = LocationDataset(
    df_target, "target_id", "target_lat", "target_long"
)

### Basic min distance flow with haversine

This flow implements the basic min distance model where we specify our parameters and the model will find the optimal results. Here are the parameters of the model:
- min_target: The minimum number of targets each enumerator is required to visit.
- max_target: The maximum number of targets each enumerator is allowed to visit.
- max_cost: The  maximum cost assignable to a surveyor to visit a single target.
- max_total_cost:  The initial maximum total cost assignable to a surveyor

In [8]:
from surveyscout.flows import basic_min_distance_flow

results = basic_min_distance_flow(
    enum_locations=enum_locations,
    target_locations=target_locations,
    min_target=5,
    max_target=30,
    max_distance=10000,
    max_total_distance=100000,
)

Optimal value:  2142243.4940421954


In [9]:
results.head()

,target_id,enum_id,value
0,T0001,E060,1.0
1,T0002,E063,1.0
2,T0003,E042,1.0
3,T0004,E061,1.0
4,T0005,E035,1.0


### Recursive min distance flow with haversine
This flow allows to recursively update parameters  until we reach a solution. 
The parameters are as follow:
- min_target: The minimum number of targets each enumerator is required to visit.
- max_target: The maximum number of targets each enumerator is allowed to visit.
- max_cost: The  maximum cost assignable to a surveyor to visit a single target.
- max_total_cost:  The initial maximum total cost assignable to a surveyor
- max_perc: The initial percentile to determine the maximum surveyor-to-target cost (default is 80).
- param_increment: The value by which the parameter bounds and percentiles are adjusted during the recursion if no solution is found (default is 5).

In [11]:
from surveyscout.flows import recursive_min_distance_flow

results_df, params = recursive_min_distance_flow(
    enum_locations=enum_locations,
    target_locations=target_locations,
    min_target=15,
    max_target=35,
    max_distance=10000,
    max_total_distance=100000,
    param_increment=5,
)

Optimal value:  2244424.350659192


In [12]:
print(params)

{'min_target': 15, 'max_target': 35, 'max_cost': 10000, 'max_total_cost': 100000}


In [8]:
results_df.head()

,target_id,enum_id,value
0,T0001,E047,1.0
1,T0002,E063,1.0
2,T0003,E042,1.0
3,T0004,E031,1.0
4,T0005,E035,1.0


# Routed Min distance flow

We will generate random enumerators and targets data in the city of Chenna in India

In [9]:
# Set a random seed for reproducibility
np.random.seed(22)
# Define the boundaries of Tamil Nadu (approximate)
min_lat = 12.9190
max_lat = 13.2400
min_lon = 80.1234
max_lon = 80.3220


# Generate 100 enumerators with random latitudes and longitudes
enumerator_data = {
    "enum_id": [f"E{i+1:03d}" for i in range(10)],  # Creates IDs like E001, E002, ...
    "enum_lat": np.random.uniform(min_lat, max_lat, 10), 
    "enum_long": np.random.uniform(min_lon, max_lon, 10), 
}

# Generate 2000 targets with random latitudes and longitudes
target_data = {
    "target_id": [
        f"T{i+1:04d}" for i in range(200)
    ],  # Creates IDs like T0001, T0002, ...
    "target_lat": np.random.uniform(min_lat, max_lat, 200),  
    "target_long": np.random.uniform(min_lon, max_lon, 200), 
}

# Create the DataFrames
df_enum = pd.DataFrame(enumerator_data)
df_target = pd.DataFrame(target_data)

# Display the shape of the created DataFrames to confirm the number of rows
print(f"Enumerators DataFrame shape: {df_enum.shape}")
print(f"Targets DataFrame shape: {df_target.shape}")

Enumerators DataFrame shape: (10, 3)
Targets DataFrame shape: (200, 3)


### Min  distance flow with OSRM
To use OSRM we will specify one more parameter:
- routing: The API used to get the route distance between points. Can be either 'google' or 'OSRM'

In [11]:
from surveyscout.flows import basic_routed_min_distance_flow

enum_locations = LocationDataset(df_enum, "enum_id", "enum_lat", "enum_long")
target_locations = LocationDataset(
    df_target, "target_id", "target_lat", "target_long"
)
results = basic_routed_min_distance_flow(
    enum_locations=enum_locations,
    target_locations=target_locations,
    min_target=5,
    max_target=30,
    max_distance=100,
    max_total_distance=1000,
    routing="osrm",
)

Optimal value:  1314.2251


In [17]:
results.head()

,target_id,enum_id,value
0,T0001,E010,1.0
1,T0002,E004,1.0
2,T0003,E006,1.0
3,T0004,E002,1.0
4,T0005,E003,1.0


### Recursive min distance Flow with OSRM
This flow has the same parameters as the recursive haversine min distance flow but with one more parameter:
- routing: The API used to get the route distance between points. Can be either 'google' or 'OSRM'

In [13]:
from surveyscout.flows import recursive_min_distance_flow

results_df, params = recursive_min_distance_flow(
    enum_locations=enum_locations,
    target_locations=target_locations,
    min_target=15,
    max_target=35,
    max_distance=100,
    max_total_distance=1000,
    param_increment=5,
    routing="osrm",
)

ConnectionError: HTTPConnectionPool(host='localhost', port=5001): Max retries exceeded with url: /table/v1/driving/-143.25417751710535,-33.87674124106154;64.01395324664284,8.786430706918452;-82.79712965082065,38.73408594703551;84.66984796413414,18.49740769289589;166.38787624227774,8.078972939441442;-90.44886833281511,-13.742136118997152;27.416640390421293,26.2609403519981;33.13509525786205,-11.234301972715343;26.010686084714422,70.51914014077437;-99.69061224937741,83.45929689018527;162.98964414611459,-20.98052661136002;-19.034863697654146,52.51050685487962;124.707122089606,5.201085555522809;71.81253911430156,12.248020996907812;-72.92269769215189,76.607394892679;112.96721509289182,-77.21350952438036;-37.257933295085536,-74.31672605372268;137.19715096001818,-86.36068846074137;29.25823414890914,59.871572198628826;137.424730267747,50.068215170973076;69.3113724279957,66.60218668442744;81.09154073507057,86.15130160189753;0.47677749361281485,53.84854155901024;164.19010850036062,-6.93371479447

In [21]:
print(params)

{'min_target': 15, 'max_target': 35, 'max_cost': 100, 'max_total_cost': 1000}


In [22]:
results.head()

,target_id,enum_id,value
0,T0001,E010,1.0
1,T0002,E004,1.0
2,T0003,E006,1.0
3,T0004,E002,1.0
4,T0005,E003,1.0
